In [123]:
import pandas as pd
import glob
import os
import datetime
import re

pd.set_option('max_rows', 10)

DATA_FOLDER = '/root/data'
DATA_FOLDER_WIN = r'D:\source\python\TnDataGen\20150401'

# convert hourly timestamp to ISO date string YYYY-mm-dd
def to_date(timestamp):
    return datetime.date.fromtimestamp(timestamp * 3600).isoformat()

files = glob.glob(os.path.join(DATA_FOLDER_WIN, 'kpidata_*.csv'))
file_name_re = r'\d{4}-\d{2}-\d{2}_\d{2}'
cell_dfs = []
nw_dfs = []

for file in [f for f in files if re.search(file_name_re, f)]:
    df = pd.read_csv(file, index_col=None, parse_dates=False, error_bad_lines=False, sep='\t')
#     df['timestamp'] = df['time_interval'].map(lambda x: pd.Timestamp(datetime.datetime.fromtimestamp(x * 3600)))
    df.drop(['utc_time_offset', 'daily_archived'], axis=1, inplace=True)
    cell_dfs.append(df)
    nw_dfs.append(df.groupby('time_interval')[df.columns[3:]].sum())

cell_hourly = pd.concat(cell_dfs)
# grouped = big.groupby([big['timestamp'].dt.date, 'g_cell_id'])
grouped = cell_hourly.groupby([cell_hourly['time_interval'].map(lambda x: to_date(x)), 'g_cell_id'])
res = grouped[cell_hourly.columns[3:]].sum()
for key, group in res.groupby(level=0):
    out_path = os.path.join(DATA_FOLDER_WIN, 'kpidata_%s.csv' % key)
    group.to_csv(out_path, sep='\t')
    print('aggregation for %s per cell written to %s' % (key, out_path))
    
nw_houly = pd.concat(nw_dfs)
out_path = os.path.join(DATA_FOLDER_WIN, 'kpidata_hourly_nw.csv')
nw_houly.to_csv(out_path, sep='\t')
print('aggregation for network hourly written to %s' % out_path)

nw_grouped = nw_houly.groupby(lambda x: to_date(x))
res = nw_grouped[nw_houly.columns[3:]].sum()
out_path = os.path.join(DATA_FOLDER_WIN, 'kpidata_daily_nw.csv')
res.to_csv(out_path, sep='\t')
print('aggregation for network daily written to %s' % out_path)

aggregation for 2015-04-01 per cell written to D:\source\python\TnDataGen\20150401\kpidata_2015-04-01.csv
aggregation for network hourly written to D:\source\python\TnDataGen\20150401\kpidata_hourly_nw.csv
aggregation for network daily written to D:\source\python\TnDataGen\20150401\kpidata_daily_nw.csv


In [110]:
res

,RRC_Setup_Attempts,RRC_Connection_Reject,RRC_Missing_CSU_Complete,NBAP_RL_Setup_Attempts,NBAP_RL_Setup_Failures,NBAP_RL_Addition_Attempts,NBAP_RL_Addition_Failures,CS_RAB_Setup_Attempts,CS_RAB_Setup_Failures,PS_RAB_Setup_Attempts,PS_RAB_Setup_Failures,Releases,Abnormal_Releases,3G_2G_HO_Attempts


In [102]:
for key, group in res.groupby(level=[0, 1, 2], as_index=False):
    print('-'.join(map(str, key)))
    print(group)

2015-4-1
                          RRC_Setup_Attempts  RRC_Connection_Reject  \
year month day g_cell_id                                              
2015 4     1   13117200                23681                    188   
               13117201                25117                    292   
               13117202                25095                    242   
               13117203                24364                    251   
               13117204                24271                    298   
...                                      ...                    ...   
               39351695                23909                    266   
               39351696                24181                    268   
               39351697                24053                    262   
               39351698                24477                    247   
               39351699                23909                    257   

                          RRC_Missing_CSU_Complete  NBAP_RL_Setup_A

In [66]:
%pfile pd.DataFrame